In [ ]:
%cd ..
%pwd

In [ ]:
from experiments.modeleval_utils import *

In [ ]:
import pickle
#output_dir = 'experiments/results'
output_dir = 'experiments/results_FS' # feature reduction
with  open(os.path.join(output_dir, 'evaluations.pickle'), 'rb') as f:
    evaluations_by_config = pickle.load(f)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
scoring = 'average_precision'
all_sampler_ranks = []
for output_dir in ['experiments/results_FS']:
    for (data, target) in [('fixed_defect_szz', 'performance'), ('bugbug_buglevel', 'performance')]: #('bugbug_buglevel', 'regression')
        for feature_type in ['traditional', 'bow']:
            #if output_dir == 'experiments/results_FS' and feature_type == 'bow':
            #    continue
            for model in models:
                try:
                    results = get_results(output_dir, data, feature_type, target, scoring, model)
                    results = results.replace({
                        None: 'No sampling',
                        'RandomOverSampler(random_state=0)': 'Random Over-Sampling',
                        'RandomUnderSampler(random_state=0)': 'Random Under-Sampling',
                        'SMOTE(random_state=0)': 'SMOTE'
                        })
                    sampler_ranks = results.groupby('param_sampler')[['mean_test_score']].max().sort_values('mean_test_score', ascending=False)
                    sampler_ranks['rank'] = np.arange(1,5)
                    sampler_ranks['model'] = np.full(4, model)
                    all_sampler_ranks.append(sampler_ranks)
                except FileNotFoundError:
                    pass
all_sampler_ranks = pd.concat(all_sampler_ranks)
all_sampler_ranks

In [ ]:
t = all_sampler_ranks.groupby('param_sampler')[['rank']].mean().sort_values('rank', ascending=True)
t.index.name = 'Sampling Method'
t = t.rename({'rank': 'Average Rank'}, axis=1)
t

In [ ]:
print(t.to_latex())

In [ ]:
scoring = 'average_precision'
best_sampler = []
for (data, target) in [('fixed_defect_szz', 'performance'), ('bugbug_buglevel', 'performance'), ('bugbug_buglevel', 'regression')]:
    for feature_type in ['traditional', 'bow']:
        for model in models:
            try:
                results = get_results(output_dir, data, feature_type, target, scoring, model)
                results = results.replace({
                    None: 'No sampling',
                    'RandomOverSampler(random_state=0)': 'Over-Sampling',
                    'RandomUnderSampler(random_state=0)': 'Under-Sampling',
                    'SMOTE(random_state=0)': 'SMOTE'
                    })
                best_sampler.append({
                    'Data': data,
                    'Feature Type': feature_type,
                    'Target': target,
                    'Model': model_names[model],
                    'Sampling':results.iloc[0]['param_sampler'],
                    'AP (val.)': results.iloc[0]['mean_test_score']
                    })
            except FileNotFoundError:
                pass

best_sampler = pd.DataFrame(best_sampler)
best_sampler = best_sampler.replace({
    'fixed_defect_szz': 'SZZ',
    'bugbug_buglevel': 'bugbug'
})
best_sampler = best_sampler.replace({
    'performance': 'perf. regressions',
    'regression': 'all regressions',
    'bow': 'bag-of-words'
})
best_sampler.set_index(pd.MultiIndex.from_frame(best_sampler[['Data', 'Target', 'Feature Type']]), inplace=True)
best_sampler = best_sampler.drop(['Data', 'Target', 'Feature Type'], axis=1)
best_sampler

In [ ]:
print(best_sampler.to_latex(index=True))

In [ ]:
best_sampler = []

scoring = 'average_precision'
for config, evaluations in evaluations_by_config.items():
    ws = config.split('_')
    feature_type = ws.pop()
    target = ws.pop()
    data = '_'.join(ws)

    amax = np.argmax([evaluation['test']['regression']['f1'] for evaluation in evaluations])
    evaluation = evaluations[amax]

    sampler = None
    if evaluation['model'] == 'tpot':
        sampler = None
    elif evaluation['best_params']:
        sampler = evaluation['best_params']['sampler']

    best_sampler.append({
        'Data': data,
        'Target': target,
        'Feature Type': feature_type,
        'Model':  model_names[evaluation['model']],
        'recall': evaluation['test']['regression']['recall'],
        'precision': evaluation['test']['regression']['precision'],
        'F1': evaluation['test']['regression']['f1'],
        'Sampling Method': str(sampler)
        })

best_sampler = pd.DataFrame(best_sampler)
best_sampler = best_sampler.replace({
                    'None': 'No sampling',
                    'RandomOverSampler(random_state=0)': 'Random Over-Sampling',
                    'RandomUnderSampler(random_state=0)': 'Random Under-Sampling',
                    'SMOTE(random_state=0)': 'SMOTE'
                    })

best_sampler = best_sampler.replace({
    'fixed_defect_szz': 'SZZ',
    'bugbug_buglevel': 'bugbug'
})
best_sampler = best_sampler.replace({
    'performance': 'perf. regressions',
    'regression': 'all regressions'
})

best_sampler.set_index(pd.MultiIndex.from_frame(best_sampler[['Data', 'Target', 'Feature Type']]), inplace=True)
best_sampler = best_sampler.drop(['Data', 'Target', 'Feature Type'], axis=1)

best_sampler    

In [ ]:
print(best_sampler.to_latex(index=True))